In [1]:
!pip install numpy pandas scikit-learn gensim torch nltk spacy keras

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Setup**

In [32]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer


import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset


import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

import gensim
import torch
import spacy
from spacy.cli import download
from collections import defaultdict

from keras.preprocessing.sequence import pad_sequences



In [33]:
download("en_core_web_sm")

columns = ["target", "ids", "date", "flag", "user", "text"]
nrows = 10000
data = pd.read_csv("/content/drive/MyDrive/Colab Data/training.1600000.processed.noemoticon.csv", names=columns, encoding = "ISO-8859-1", nrows=nrows)
data.head()
print(data.head())

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   target         ids                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the 

In [4]:
decode_map = {0: "NEGATIVE", 2: "NEUTRAL", 4: "POSITIVE"}
def decode_sentiment(label):
    return decode_map[int(label)]

In [5]:
# Preprocess
nltk.download("stopwords")
nltk.download('punkt_tab')

TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

# data.target = data.target.apply(lambda x: decode_sentiment(x))
if data.target.dtype != object:  # If not object (string) type, then decode
    data.target = data.target.apply(lambda x: decode_sentiment(x))


data.text.apply(lambda x: preprocess(x))
train, test = train_test_split(data, test_size = 0.2, shuffle=True)

documents = [text.split() for text in train.text]
print(documents)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


[['Today', 'is', 'not', 'my', 'day.', "I've", 'spilled', 'my', 'water', 'twice.', 'Dropping', 'the', 'water', 'bottle', 'once.', "It's", 'not', 'broken', 'yet,', 'but', "I'm", 'expecting', 'it.'], ['@s13_eisbaer', 'your', 'face', 'is', 'a', 'D90.'], ['First', 'time', 'to', 'go', 'negative', 'in', 'my', 'bank', 'account!!', 'i', 'am', 'not', 'a', 'happy', 'camper!!', 'what', 'was', 'i', 'thinking!'], ['@InspiredWriting', 'I', 'love', 'how', 'I', 'can', '-', 'could', '-', 'easily', 'read', 'and', 'tweed', 'while', 'working.', 'With', 'twitter.com', "I'm", 'reloading', 'all', 'the', 'time'], ['has', 'a', 'lot', 'on', 'her', 'mind.', 'I', 'needs', 'to', 'make', 'some', 'moneys.', 'And', 'ideas.', 'Nowhere', 'is', 'hiring!'], ['Talked', 'to', '@cameren', '@jfarrell', '&amp;', '@laughyladylora', '&amp;', 'now', 'i', 'wish', 'i', 'could', 'be', 'there', 'with', 'them', 'at', 'Sixx', 'nightclub', 'instead', "i'm", 'stuck', 'working.'], ['@BANANARCHY', 'empires', 'is', 'the', 'only', 'band/song

In [6]:
w2v = gensim.models.word2vec.Word2Vec(
    vector_size=W2V_SIZE,
    window=W2V_WINDOW,
    epochs=W2V_EPOCH,
    min_count=W2V_MIN_COUNT
)
w2v.build_vocab(documents)
w2v.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

(1697738, 3486080)

# **Tokenization**

In [7]:
vocab = defaultdict(int)
count = 0

def fit_tokenizer(text: str):
    global count
    tokens = nltk.tokenize.word_tokenize(text)
    tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]
    for token in tokens:
        if token not in vocab:
            vocab[token] = count
            count += 1

def text_to_sequence(text: str):
    tokens = nltk.tokenize.word_tokenize(text)
    tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]
    seq = [vocab[token] for token in tokens]
    return seq

def texts_to_sequences(texts: list):
    result = []
    for text in texts:
        result.append(text_to_sequence(text))
    return result

In [8]:
# Tokenize
for text in train.text:
    fit_tokenizer(text)


x_train = pad_sequences(texts_to_sequences(train.text), maxlen=300)
x_test = pad_sequences(texts_to_sequences(test.text), maxlen=300)

In [9]:
labels = train.target.unique().tolist()
labels.append("NEUTRAL")
labels

['NEGATIVE', 'NEUTRAL']

In [31]:
encoder = LabelEncoder()
encoder.fit(train.target.tolist())

y_train = encoder.transform(train.target.tolist())
y_test = encoder.transform(test.target.tolist())

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [34]:
train_ds = TensorDataset(torch.tensor(x_train, dtype=torch.long), torch.tensor(y_train, dtype=torch.long))
test_ds = TensorDataset(torch.tensor(x_test, dtype=torch.long), torch.tensor(y_test, dtype=torch.long))

# **Model Architecture**

In [61]:
# embedding layer
embedding_matrix = np.zeros((len(vocab) + 1, W2V_SIZE))
for word, i in vocab.items():
    if word in w2v.wv:
        embedding_matrix[i] = w2v.wv[word]

In [62]:
embedding_matrix = torch.tensor(embedding_matrix, dtype=torch.float32)
embedding_layer = nn.Embedding.from_pretrained(embedding_matrix, freeze=True)

In [99]:
class SentimentAnalysis(nn.Module):
    def __init__(self, embedding_layer):
        super(SentimentAnalysis, self).__init__()
        self.embedding = embedding_layer
        self.lstm = nn.LSTM(W2V_SIZE, 100, batch_first=True)
        self.fc = nn.Linear(100, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

model = SentimentAnalysis(embedding_layer)

In [27]:
# print(x_train)
print(y_train)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [64]:
batch_size = 32
epochs = 8
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

In [100]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(epochs):
    model.train()  # Set model to training mode
    for batch in train_loader:
        input_batch, truth_batch = batch
        # Forward pass
        input_batch = input_batch.to(torch.int64)
        truth_batch = truth_batch.to(torch.float32)

        outputs = model(input_batch)
        loss = nn.CrossEntropyLoss()(outputs, truth_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])


# **Testing**

In [108]:
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)
model.eval()
correct = 0
total = len(x_test)

In [109]:
with torch.no_grad():
    for input_batch, truth_batch in test_loader:
        input_batch = input_batch.to(torch.int64)
        truth_batch = truth_batch.to(torch.float32)

        outputs = model(input_batch)

        preds = (outputs >= 0.5).float()
        correct += (preds == truth_batch).sum().item()
print(correct/total)

0.9975


In [124]:
seq = text_to_sequence("")
seq = pad_sequences([seq], maxlen=300)
seq = torch.tensor(seq, dtype=torch.long)
print(model(seq))


tensor([[0.4829]], grad_fn=<SigmoidBackward0>)
